In [362]:
import serial

#import matplotlib as plt
import sys
import visa
from LQCE.zvl13 import *
from LQCE.ncs513 import *
from LQCE.N5173B import *
from LQCE.awg33500 import *
import matplotlib.pyplot as plt
import numpy as np
import math
import matplotlib.cm as cm
from matplotlib.backends.backend_pdf import PdfPages
import time
import scipy.signal
import scipy.optimize
%matplotlib 
#%matplotlib inline


Using matplotlib backend: Qt5Agg


# __AWG Init cell__
## ch1 - pulse with offset
## ch2 - dc offset
## sync - trigger out for n5173b

In [699]:
def awg_init_burst_mode():  
    awg.set_modulation_source("INT")
    awg.set_waveform_type(1,"PULSe")
    awg.set_waveform_type(2,"DC")
    awg.set_voltage_offset(2, 0)
    awg.set_voltage_offset(1, 0.05)
    awg.set_pulse_width(1,1e-6)#set width 100ns
    awg.set_pulse_period(1,1e-4)#set period 1ms
    awg.set_pulse_both_edge(1, 20e-9)
    awg.set_sync_source("CH1")
    awg.set_sync_mode(1, "CARRier")
    awg.set_burst_state(1,"ON")
    awg.set_burst_mode(1, "TRIGgered")
    awg.set_burst_count(1, 10000)
    awg.set_burst_trigger_source(1, "BUS")
    awg.set_sync_output("ON")
    awg.set_output(1, "ON")
    awg.set_output(2, "ON")
def awg_init(mixer_calibration_result):  
    awg.set_modulation_source("INT")
    awg.set_waveform_type(1,"PULSe")
    awg.set_waveform_type(2,"DC")
    awg.set_voltage_offset(2, mixer_calibration_result[1])
    awg.set_voltage_offset(1, 0.05 + mixer_calibration_result[0])
    awg.set_pulse_period(1,13e-6)
    awg.set_voltage(1,100e-3)
    awg.set_pulse_width(1,9e-6)
    awg.set_pulse_period(1,13e-6)
    awg.set_pulse_both_edge(1, 20e-9)
    awg.set_sync_source("CH1")
    awg.set_sync_mode(1, "CARRier")
    awg.set_sync_output("ON")
    awg.set_output(1, "ON")
    awg.set_output(2, "ON")
def awg_init_custom(mixer_calibration_result, readout_width, period):  
    awg.set_modulation_source("INT")
    awg.set_waveform_type(1,"PULSe")
    awg.set_waveform_type(2,"DC")
    awg.set_voltage_offset(2, mixer_calibration_result[1])
    awg.set_voltage_offset(1, 0.05 + mixer_calibration_result[0])
    awg.set_voltage(1,100e-3)
    awg.set_pulse_period(1,period)
    awg.set_pulse_width(1,readout_width)
    awg.set_pulse_period(1,period)
    awg.set_pulse_both_edge(1, 20e-9)
    awg.set_sync_source("CH1")
    awg.set_sync_mode(1, "CARRier")
    awg.set_sync_output("ON")
    awg.set_output(1, "ON")
    awg.set_output(2, "ON")
def awg_init_custom_2ch(mixer_calibration_result, readout_width, period):  
    awg.set_modulation_source("INT")
    awg.set_waveform_type(1,"PULSe")
    awg.set_waveform_type(2,"PULSe")
    awg.set_voltage_offset(2, 0.05 + mixer_calibration_result[1])
    awg.set_voltage_offset(1, 0.05 + mixer_calibration_result[0])
    awg.set_voltage(2,100e-3)
    awg.set_voltage(1,100e-3)
    
    awg.set_pulse_period(1,period)
    awg.set_pulse_width(1,readout_width)
    awg.set_pulse_period(1,period)
    awg.set_pulse_both_edge(1, 20e-9)
    
    awg.set_pulse_period(2,period)
    awg.set_pulse_width(2,readout_width)
    awg.set_pulse_period(2,period)
    awg.set_pulse_both_edge(2, 20e-9)  
    
    awg.set_sync_source("CH1")
    awg.set_sync_mode(1, "CARRier")
    awg.set_sync_output("ON")
    awg.set_output(1, "ON")
    awg.set_output(2, "ON") 
    
def awg_base_init():  
    awg.set_waveform_type(1,"DC")
    awg.set_waveform_type(2,"DC")
    awg.set_voltage_offset(1, 0)
    awg.set_voltage_offset(2, 0)
    awg.set_sync_output("OFF")
    awg.set_output(1, "ON")
    awg.set_output(2, "ON")
def awg_set_voltage_with_offset(value, offset):
    awg.set_voltage_offset(1, offset[0] + value/2)
    awg.set_voltage(1, value)
def awg_set_delay(delay):
    awg.set_phase_units("SEC")
    awg.write("SOURce1:PHASe {}".format(-100e-9 - delay*1e-9))
    
#n5173b part    
def gen_init_pulse_mode(freq, pulse_start_len):
    gen.set_output_state("OFF")
    gen.set_frequency(freq)
    gen.set_alc_state("OFF")
    gen.set_modulation_state("ON")
    gen.set_pulse_out_state("ON")
    gen.set_power(-5)
    gen.set_trigger_source("EXTernal")
    gen.set_trigger_external_source("PULSe")
    gen.set_pulse_mode("PTRain")
    gen.set_pulse_train_ontime(pulse_start_len)
    gen.set_pulse_train_offtime(20e-9)
    gen.set_pulse_train_repetition(1)
    gen.set_pulse_train_trigger("TRIGgered")
    gen.set_pulse_internal_step_delay(10e-9)
    gen.set_pulse_internal_delay(0)
    gen.set_output_state("ON")
def gen_init_ramsey_mode(freq, pulse_start_len, delay = 10e-9):
    gen.set_output_state("OFF")
    gen.set_frequency(freq)
    gen.set_alc_state("OFF")
    gen.set_modulation_state("ON")
    gen.set_pulse_out_state("ON")
    gen.set_power(-15)
    gen.set_trigger_source("EXTernal")
    gen.set_trigger_external_source("PULSe")
    gen.set_pulse_mode("PTRain")
    #gen.set_pulse_train_ontime(pulse_start_len)
    #gen.set_pulse_train_offtime(20e-9 + delay, 20e-9)
    #gen.set_pulse_train_repetition(1)
    gen.write(":PULM:INTernal:TRAin:ONTime {},{}".format(pulse_start_len, pulse_start_len)) #:PULM:INTernal:TRAin:ONTime <value>,<value>
    gen.write(":PULM:INTernal:TRAin:OFFTime {},{}".format(10e-9 + delay, 20e-9)) #:PULM:INTernal:TRAin:OFFTime <value>,<value>
    gen.write(":PULM:INTernal:TRAin:REPetition {},{}".format(1,1)) #:PULM:INTernal:TRAin:REPetition <value>,<value>
    gen.set_pulse_train_trigger("TRIGgered")
    gen.set_pulse_internal_step_delay(10e-9)
    gen.set_pulse_internal_delay(0)
    gen.set_output_state("ON")

  
    
#mixer calibration part    
def find_minimum(x):
    awg.set_voltage_offset(2, x[1])
    awg.set_voltage_offset(1, x[0])
    time.sleep(0.1)
    nwa.trig_meas()
    temp_data = nwa.get_data(3)
    data_averaged = np.average(temp_data[0],0)
    #temp_slice = data_averaged[1]
    return (data_averaged[1])

def calibrate(power, freq):
    nwa.set_standard_config({"power": power,"nop":2, "cf_span":[freq, 2], "bandwidth":10, "meas_counts":10})
    nwa.set_output_state("ON")
    awg_base_init()
    #gen.set_output_state("OFF")  
    awg.set_output(1, "ON")
    awg.set_output(2, "ON") 
    
    calibrate_res = scipy.optimize.minimize(find_minimum,(0.05,-0.05), method='Nelder-Mead', options = {"maxiter": 50, 'xtol': 1e-2, 'disp': True}, bounds = ((-0.1,0.1), (-0.1,0.1)))
    print(calibrate_res)
    return (calibrate_res.x, calibrate_res.fun) 

def get_data_reim(data_frames_count):
    curves = [0]*data_frames_count
    out_data = []
    out_phase = []
    out_re = []
    out_im = []
    for i in range(data_frames_count):
        curves[i] = nwa.query("CALCulate1:DATA:NSWeep? SDATa, {}".format(i+1))
        np_d = np.fromstring(curves[i], sep = ',')
        temp_data = []*int(len(np_d)/2)
        temp_phase = []*int(len(np_d)/2)
        temp_re = []*int(len(np_d)/2)
        temp_im = []*int(len(np_d)/2)
        for j in range(int(len(np_d)/2)):
            temp_data.append(20*math.log(math.sqrt(np_d[2*j]**2+np_d[2*j+1]**2),10))
            temp_phase.append(math.atan2(np_d[2*j+1],np_d[2*j]))
            temp_re.append(np_d[2*j])
            temp_im.append(np_d[2*j+1])
        out_re.append(temp_re)
        out_im.append(temp_im)
    return (out_re, out_im)    

# __INIT__

In [677]:
nwa = zvl13("TCPIP0::192.168.0.73::INSTR")
awg = awg33500("TCPIP0::192.168.0.80::INSTR")
gen = N5173B_gen("TCPIP0::192.168.0.78::INSTR")
ncs = Nsk_current_source("COM5")
ncs.enable_channel(3)
ncs.set_output_state(3,1)
ncs.set_range(3,"1mA")
ncs.set_current(3,110e-6)

b'\xc1\xe3\xcd\xaf\xe68\xef\xef\xef:'

In [68]:
#gen.set_pulse_train_ontime(100e-9)
gen.set_power(-10)
#gen.set_output_state("ON")
#awg_set_delay(100)

In [124]:
gen.set_output_state("OFF")
nwa.set_output_state("ON")
awg_init(cal_result[0])
nwa.set_standard_config({"power":-20,"nop":2, "cf_span":[7.055111e9, 2], "bandwidth":10, "meas_counts":40})
nwa.set_timeout(500000)
dt_sin = []
plt.ion()
fig = plt.figure()

pd_counts = 5

for i in range(pd_counts):
    awg_set_voltage_with_offset((i+1)*20e-3, cal_result[0])
    time.sleep(0.1)
    nwa.trig_meas()
    temp_data = nwa.get_data(40)
    data_sin = np.average(temp_data[0],0)
    dt_sin.append(data_sin[0])
    plt.title("{}/{}counts".format(i+1, pd_counts))
    plt.plot(dt_sin)
    plt.grid()
    fig.canvas.draw()
    fig.canvas.flush_events()


In [655]:

awg_init(cal_result[0])

# __Mixer Calibration__

In [701]:
cal_result = calibrate(-10, 7.058685e9)

 final_simplex: (array([[ 0.05478928, -0.02544861],
       [ 0.05478928, -0.02544861],
       [ 0.05478928, -0.02544861]]), array([-95.62821116, -86.72854893, -86.41919817]))
           fun: -95.62821116458538
       message: 'Maximum number of iterations has been exceeded.'
          nfev: 126
           nit: 50
        status: 2
       success: False
             x: array([ 0.05478928, -0.02544861])


In [290]:
# |DP\__delay(loop)__|RO|
                

gen.set_modulation_state(0)
gen.set_frequency(6.61e9)
gen.set_output_state("ON")
gen.set_power(-5)

awg_init(cal_result[0])
#gen_init_pulse_mode(6.54e9, 1000-9)#at this moment gen wait a trigger to start pulse
awg_set_delay(1000)#in ns| for ex. (100) = 100ns (with offset 100ns = 200ns)
nwa.set_standard_config({"power":-20,"nop":2, "cf_span":[7.057069e9, 2], "bandwidth":10, "meas_counts":10})

decay_step = 100
decay_counts = 30

decay_data_result = []
decay_x_axis = []
plt.ion()
fig = plt.figure()
nwa.set_timeout(500000)
for i in range(decay_counts):
    #awg_set_delay(1000 + i*decay_step)
    
    gen.set_frequency(6.61e9 + 0.1e6*i)
    time.sleep(0.5)
    nwa.trig_meas()
    temp_data = nwa.get_data(10)
    
    temp_dB = np.average(temp_data[0],0)
    decay_data_result.append(temp_dB[1])
    decay_x_axis.append(20 + i*decay_step)
    plt.ylabel("dB")
    plt.xlabel("delay, ns")
    plt.title("Decay {}/{}counts".format(i+1, decay_counts))
    plt.plot(decay_x_axis, decay_data_result, 'r')
    
    fig.canvas.draw()
    fig.canvas.flush_events()


# __Decay__

In [190]:
# |DP\__delay(loop)__|RO|


gen.set_modulation_state(0)
gen.set_frequency(6.64e9)#6533500000.0
gen.set_output_state("ON")
gen.set_power(-15)

awg_init(cal_result[0])
gen_init_pulse_mode(6.64e9, 45e-9)#at this moment gen wait a trigger to start pulse
awg_set_delay(20)#in ns| for ex. (100) = 100ns (with offset 100ns = 200ns)
nwa.set_standard_config({"power":-18,"nop":2, "cf_span":[7.058722e9, 2], "bandwidth":100, "meas_counts":300})

decay_step = 4
decay_counts = 500

decay_data_result = []
decay_x_axis = []
plt.ion()
fig = plt.figure()
nwa.set_timeout(500000)
for i in range(decay_counts):
    awg_set_delay(47 + i*decay_step)
    time.sleep(1)
    nwa.trig_meas()
    temp_data = nwa.get_data(20)
    
    temp_dB = np.average(temp_data[0],0)
    decay_data_result.append(temp_dB[1])
    decay_x_axis.append(i*decay_step)
    plt.ylabel("dB")
    plt.xlabel("delay, ns")
    plt.title("Decay {}/{}counts".format(i+1, decay_counts))
    plt.plot(decay_x_axis, decay_data_result, 'r')
    
    fig.canvas.draw()
    fig.canvas.flush_events()


# __Decay save data__

In [191]:
np.save("E:\\MEASUREMENTS\\10mK_BlueFors_BFLD400\\RUN33\\run33\\decay_data_result_4ns_47nsDr_500nop_100av", decay_data_result)

In [202]:
#nwa.set_output_state("ON")
ncs.set_current(3,103e-6)
#gen.set_output_state("OFF")
awg.set_waveform_type(1,"DC")
awg.set_voltage_offset(1, cal_result[0][1])
awg.set_voltage_offset(1, 0.05 + cal_result[0][0])
awg.set_voltage(1,100e-3)

In [704]:
#nwa.set_power(-20)
ncs.set_current(3,110e-6)
gen.set_frequency(10.635e9)
gen.set_modulation_state(0)

# __Rabi__
### _DP(in loop)__delay(fixed)__RO(fixed)_

In [705]:
#awg_init(cal_result[0])
awg_init_custom_2ch(cal_result[0], 9e-6, 15e-6)
gen_init_pulse_mode(6.641e9, 20e-9)#at this moment gen wait a trigger to start pulse
awg_set_delay(20)#in ns| for ex. (100) = 100ns (with offset 100ns = 200ns)

average_factor = 360
nwa.set_standard_config({"power":-10,"nop":2, "cf_span":[7.058685e9, 2], "bandwidth":10, "meas_counts":average_factor})

gen.set_power(-6)

rabi_step = 4
rabi_counts = 401
rabi_data_result = []
rabi_phase_result = []
rabi_x_axis = []
plt.ion()
fig = plt.figure()
ax1 = fig.add_subplot(2,1,1)
ax2 = fig.add_subplot(2,1,2)
plt.setp(ax1.get_xticklabels(), visible=False)
fig.set_size_inches(10, 6)

nwa.set_timeout(500000)
for i in range(rabi_counts):
    #awg.set_sync_output("OFF")
    #gen.set_output_state("OFF")
    time.sleep(1)
    #gen.set_pulse_train_ontime(500e-9 + i*1e-9*rabi_step)
    gen.write(":PULM:INTernal:TRAin:ONTime {:.3e}".format(20e-9 + i*1e-9*rabi_step))
    time.sleep(2)
    awg_set_delay(20 + i*rabi_step)
    
    
    #awg.set_sync_output("ON")
    #gen.set_output_state("ON")
    time.sleep(1)
    nwa.trig_meas()
    #temp_data = nwa.get_data(average_factor)
    temp_data = get_data_reim(average_factor)
    
    temp_dB = np.average(temp_data[0],0)
    temp_Phase = np.average(temp_data[1],0)
    
    rabi_data_result.append(temp_dB[0])
    rabi_phase_result.append(temp_Phase[0])
    rabi_x_axis.append(20 + i*rabi_step)
    
    ax2.set_xlabel("delay, nS")
    ax1.plot(rabi_x_axis, rabi_data_result, 'b--')
    ax1.set_ylabel("Re")
    ax2.plot(rabi_x_axis, rabi_phase_result, 'g--')
    ax2.set_ylabel("Im")
    ax1.set_title("Rabi {}/{}".format(i+1, rabi_counts))
    
    
    #plt.title("Rabi {}/{}counts".format(i+1, rabi_counts))
    #plt.ylabel("dB")
    #plt.xlabel("dt width, ns")
    
    #plt.plot(decay_x_axis, decay_data_result, 'r--')
    
    fig.canvas.draw()
    fig.canvas.flush_events()


KeyboardInterrupt: 

In [50]:
plt.ion()
fig = plt.figure()
data_angle = []
for i in range(len(rabi_phase_result)):
    data_angle.append(rabi_phase_result[i]*180/(np.pi))
plt.plot(data_angle)


# _save Rabi_

In [706]:
#np.save("E:\\MEASUREMENTS\\10mK_BlueFors_BFLD400\\RUN33\\run33\\rabi_data_result_4ns_20nsDr_360nop_100av", decay_data_result)
np.save("E:\\MEASUREMENTS\\10mK_BlueFors_BFLD400\\RUN33\\run33\\precise10Hz_rabi_re_4ns_20nsDr_229nop_360av", rabi_data_result)
np.save("E:\\MEASUREMENTS\\10mK_BlueFors_BFLD400\\RUN33\\run33\\precise10Hz_rabi_im_4ns_20nsDr_229nop_360av", rabi_phase_result)

# __Ramsey__
### _min step 20ns_

In [ ]:
awg_init_custom(cal_result[0], 9e-6,15e-6) #pulse width & period
gen_init_ramsey_mode(6.651e9, 28e-9, delay = 40e-9) # freq | pulse width | delay(min 10ns)
#6.64e9 - qubit freq
awg_set_delay(110)#in ns| for ex. (100) = 100ns (with offset 100ns = 200ns)

average_factor = 60
nwa.set_standard_config({"power":-10,"nop":2, "cf_span":[7.058685e9, 2], "bandwidth":10, "meas_counts":average_factor})

gen.set_power(-6)

ramsey_step = 4
ramsey_counts = 80

ramsey_data_result = []
ramsey_phase_result = []
ramsey_x_axis = []
plt.ion()
fig = plt.figure()
ax1 = fig.add_subplot(2,1,1)
ax2 = fig.add_subplot(2,1,2)
plt.setp(ax1.get_xticklabels(), visible=False)
fig.set_size_inches(10, 6)

nwa.set_timeout(500000)

for i in range(ramsey_counts):

    time.sleep(1)
    #gen.set_pulse_train_ontime(500e-9 + i*1e-9*rabi_step)
    #gen.write(":PULM:INTernal:TRAin:ONTime {:.3e}".format(20e-9 + i*1e-9*rabi_step))
    gen.write(":PULM:INTernal:TRAin:OFFTime {:.4e},{:.4e}".format(20e-9 + ramsey_step*i*1e-9, 40e-9))
    
    time.sleep(2)
    awg_set_delay(20 + i*ramsey_step)
    
    
    time.sleep(1)
    nwa.trig_meas()
    #temp_data = nwa.get_data(average_factor)
    temp_data = get_data_reim(average_factor)
    
    temp_dB = np.average(temp_data[0],0)
    temp_Phase = np.average(temp_data[1],0)
    
    ramsey_data_result.append(temp_dB[0])
    ramsey_phase_result.append(temp_Phase[0])
    ramsey_x_axis.append(20 + i*ramsey_step)
    #plt.title("Rabi {}/{}counts".format(i+1, ramsey_counts))
    ax2.set_xlabel("delay, nS")
    ax1.plot(ramsey_x_axis, ramsey_data_result, 'b--')
    ax1.set_ylabel("Re")
    ax2.plot(ramsey_x_axis, ramsey_phase_result, 'g--')
    ax2.set_ylabel("Im")
    ax1.set_title("Ramsey {}/{}".format(i+1, ramsey_counts))
    
    
    fig.canvas.draw()
    fig.canvas.flush_events()


In [221]:
plt.ion()
fig = plt.figure()
plt.plot(ramsey_x_axis, ramsey_data_result, 'r--')

# __Ramsey__
## _save data to npy file_

In [574]:
np.save("E:\\MEASUREMENTS\\10mK_BlueFors_BFLD400\\RUN33\\run33\\ramsey_3rdtone_+7dbm_re_4ns_27nsDr_151nop_200av", ramsey_data_result)
np.save("E:\\MEASUREMENTS\\10mK_BlueFors_BFLD400\\RUN33\\run33\\ramsey_3rdtone_+7dbm_im_4ns_27nsDr_151nop_200av", ramsey_phase_result)

In [489]:
im = np.load("E:\\MEASUREMENTS\\10mK_BlueFors_BFLD400\\RUN33\\run33\\ramsey_im_4ns_25nsDr_151nop_200av.npy")
re = np.load("E:\\MEASUREMENTS\\10mK_BlueFors_BFLD400\\RUN33\\run33\\ramsey_re_4ns_25nsDr_151nop_200av.npy")

dB_rabi = np.load("E:\\MEASUREMENTS\\10mK_BlueFors_BFLD400\\RUN33\\run33\\rabi_data_result_4ns_20nsDr_360nop_100av.npy")

plt.ion()
fig = plt.figure()
x_axis = np.linspace(20e-9, 1460e-9, 360)
#plt.plot(x_axis, re)


def fit_func(xi,ai, bi, of, decay, phase):
    return (ai*np.sin(bi*xi + phase)*np.exp(decay*xi) + of)


def fit_decay_rabi(xi, ai, bi, decay):
    return ai*np.sin(bi * xi)*np.exp(decay)

#x = np.linspace(0, 10*np.pi, 1000)

ydata = dB_rabi #5*np.sin(5*x) + np.random.normal(size=1000) + 5   
xdata = x_axis #np.linspace(0, 10*np.pi, 1000)   

params, params_covariance  = scipy.optimize.curve_fit(fit_func, xdata, ydata, p0 = [20 , 70e6, -30, -50000000, np.pi/8])

print(params)

plt.plot(x_axis, fit_func(xdata, params[0], params[1], params[2], params[3], params[4]))
plt.plot(x_axis, ydata)



[ 1.96291466e-01 -6.78659402e+07 -2.74795274e+01 -1.49010848e+06
  4.96687866e+00]


In [477]:
dec = np.load("E:\\MEASUREMENTS\\10mK_BlueFors_BFLD400\\RUN33\\run33\\ramsey_im_4ns_27nsDr_151nop_200av.npy")
#здесь подгрузить данные decay нужно
plt.ion()
fig = plt.figure()
x_axis = np.linspace(47e-9, 2047e-9, 151)
#plt.plot(x_axis, re)


def fit_func(xi,ai, of, decay):
    return (ai*np.exp(decay*xi) + of)

#x = np.linspace(0, 10*np.pi, 1000)

ydata = dec #5*np.sin(5*x) + np.random.normal(size=1000) + 5   
xdata = x_axis #np.linspace(0, 10*np.pi, 1000)   

params, params_covariance  = scipy.optimize.curve_fit(fit_func, xdata, ydata, p0 = [0.2 , -30, -0.02])

print(params)

plt.plot(x_axis, fit_func(xdata, params[0], params[1], params[2]))
plt.plot(x_axis, ydata)



[ 3.74394485e+02 -3.74409314e+02  4.25927569e-03]
Figure(640x480)


In [110]:
%matplotlib 
nwa.set_output_state("ON")
nwa.set_standard_config({"power":-20,"nop":101, "cf_span":[7.058422e9, 8e6], "bandwidth":100, "meas_counts":4})

ncs.enable_channel(3)
ncs.set_output_state(3,1)
ncs.set_range(3,"1mA")
#ncs.set_current(3,0)
nwa.set_timeout(200000)


plt.ion()

fig = plt.figure()
z_upd = []


for i in range(4):
    ncs.set_current(3, 50e-6+i*1e-6)#10uA step
    nwa.trig_meas()
    x_axis = nwa.get_X_axis_values()   
    data = nwa.get_data(int(nwa.get_meas_counts()))
    x_upd = x_axis#[]
    y_upd = []
    nx = 101#x_upd.shape[0]
    no_labels = 5
    step_x = int(nx/(no_labels -1))
    x_positions = np.arange(0,nx,step_x)
    x_labels = x_upd[::step_x]
    plt.yticks(x_positions, x_labels)
    
    z_upd.append(data[0][0])
    y_upd.append(i)
    plt.imshow(np.transpose(z_upd),interpolation='nearest', aspect='auto', cmap=cm.RdBu, origin = 'lower')
    #nwa.set_timeout(1000)
    #plt.plot(x_axis, np.average(data[0],0))
    #line1.set_ydata(data[0][i][50])
    fig.canvas.draw()
    fig.canvas.flush_events()
plt.colorbar()

Using matplotlib backend: Qt5Agg


In [434]:
gen.set_modulation_state(0)
gen.set_frequency(6.631e9)
gen.set_output_state("ON")
gen.set_power(-6)
ncs.set_current(3,110e-6)

b'\xc1\xe3\xcd\xaf\xe68\xef\xef\xef:'

In [548]:
ncs.set_current(3,110e-6)

b'\xc1\xe3\xcd\xaf\xe68\xef\xef\xef:'

In [554]:
gen.set_output_state("ON")
gen.set_frequency(6.641e9)

# __TWO TONE__

In [691]:
%matplotlib 
#nwa.set_output_state("ON")
nwa.set_standard_config({"power":-40,"nop":801, "cf_span":[7.0555071e9, 12e6], "bandwidth":10000, "meas_counts":1})

#ncs.enable_channel(3)
#ncs.set_output_state(3,1)
#ncs.set_range(3,"1mA")
#ncs.set_current(3,0)
nwa.set_timeout(200000)


plt.ion()

fig = plt.figure()
z_upd = []


for i in range(240):
    ncs.set_current(3, 40e-6+i*1e-6)#10uA step
    nwa.trig_meas()
    x_axis = nwa.get_X_axis_values()   
    data = nwa.get_data(int(nwa.get_meas_counts()))
    x_upd = x_axis#[]
    y_upd = []
    nx = 101#x_upd.shape[0]
    no_labels = 5
    step_x = int(nx/(no_labels -1))
    x_positions = np.arange(0,nx,step_x)
    x_labels = x_upd[::step_x]
    plt.yticks(x_positions, x_labels)
    
    z_upd.append(data[0][0])
    y_upd.append(i)
    plt.imshow(np.transpose(z_upd),interpolation='nearest', aspect='auto', cmap=cm.RdBu)
    #nwa.set_timeout(1000)
    #plt.plot(x_axis, np.average(data[0],0))
    #line1.set_ydata(data[0][i][50])
    fig.canvas.draw()
    fig.canvas.flush_events()
plt.colorbar()

Using matplotlib backend: Qt5Agg


In [692]:
np.save("E:\\MEASUREMENTS\\10mK_BlueFors_BFLD400\\RUN33\\run33\\sweetspot_-40dBm_10000Hz_1av_12MHz_801nop_7p055071G_240points", np.transpose(z_upd))


In [547]:
nwa.set_output_state("ON")
nwa.set_standard_config({"power":-10,"nop":5,"averages": 1, "cf_span":[7.058685e9, 8e6], "bandwidth":100, "meas_counts":5})
nwa.set_timeout(500000)
gen = N5173B_gen("TCPIP0::192.168.0.78::INSTR")
gen.set_modulation_state(0)
gen.set_frequency(6.4e9)
gen.set_output_state("ON")
gen.get_output_state()

gen.set_power(-6)

gen_freq_list = 135
plt.ion()

fig = plt.figure()

pow_slice = []
phase_slice = []

for j in range(gen_freq_list):
    

    gen.set_frequency(6.4e9 + j*5e6, "Hz")
    time.sleep(1)
    temp_data = []
    onetone_data = []
    phase_data = []
    
    for i in range(20):
        
        
        ncs.set_current(3,100e-6+1e-6*i)#10uA step
        time.sleep(0.02)
        nwa.trig_meas()
        x_axis = nwa.get_X_axis_values()  
        temp_data = nwa.get_data(1)
        
        temp_dB = np.average(temp_data[0],0)
        #temp_Phase = temp_data[1]

        #decay_data_result.append(temp_dB[0]) 
        
        #data = nwa.get_data(int(nwa.get_meas_counts()))
        
        x_upd = x_axis#[]
        y_upd = []
        #nx = 101#x_upd.shape[0]
        #no_labels = 5
        #step_x = int(nx/(no_labels -1))
        #x_positions = np.arange(0,nx,step_x)
        #x_labels = x_upd[::step_x]
        #plt.yticks(x_positions, x_labels)
        
        #z_upd.append(data[0][0])
        #y_upd.append(i)
        nwa_slice = temp_data[0][0]
        nwa_slice_phase = temp_data[0][0]
        onetone_data.append(nwa_slice[2])
        
        #onetone_data.append(temp_data[0])
        phase_data.append(nwa_slice_phase[2])
        #plt.imshow(np.transpose(z_upd),interpolation='nearest', aspect='auto', cmap=cm.RdBu)
        #nwa.set_timeout(1000)
        #plt.plot(x_axis, np.average(data[0],0))
        #line1.set_ydata(data[0][i][50])
        #fig.canvas.draw()
        #fig.canvas.flush_events()
    #print("Current freq - {}".format((j+1)*20e6+5e9)    
    pow_slice.append(onetone_data)
    phase_slice.append(phase_data)
    plt.imshow(pow_slice,interpolation='nearest', aspect='auto', cmap=cm.bone, origin = 'lower')
    #plt.imshow(phase_slice,interpolation='nearest', aspect='auto', cmap=cm.bone, origin = 'lower')
    plt.title("Two tone spectroscopy - {}/{}".format((j+1), gen_freq_list))
    fig.canvas.draw()
    fig.canvas.flush_events()        
plt.colorbar()

KeyboardInterrupt: 

In [332]:
#fig = plt.figure()
#plt.imshow(pow_slice,interpolation='nearest', aspect='auto', cmap=cm.bone, origin = 'lower')
6.3e9 + 5e6*71

6655000000.0

# __Save Two tone Spectroscopy__

In [519]:
np.save("E:\\MEASUREMENTS\\10mK_BlueFors_BFLD400\\RUN33\\run33\\two_3rd_+7dbm_6.4e9_135p_90e-6_1e-6_40_5av_probe_0dbm_drive_-6dbm", pow_slice)
#fig = plt.figure()
#data_2_tone = np.load("E:\\MEASUREMENTS\\10mK_BlueFors_BFLD400\\RUN33\\run33\\two_tone_6.3e9_180p_90e-6_1e-6_40_200av.npy")
#plt.imshow(data_2_tone, interpolation='nearest', aspect='auto', cmap=cm.bone, origin = 'lower')

In [104]:
data_dt = []
for i in range(len(pow_slice)):
    data_temp = pow_slice[i]
    data_dt.append(data_temp[20])
np.max(data_dt)
np.argmax(data_dt)


151

In [105]:
plt.ion()

fig = plt.figure()
plt.imshow(pow_slice,interpolation='nearest', aspect='auto', cmap=cm.bone, origin = 'lower')

In [343]:
for i in range(11):
    #print(20e-9 + i*20e-9)
    #print(format(20e-9 + i*20e-9, "5.3f"))
    print("{:.1e}".format(20e-9 + i*20e-9))

2.0e-08
4.0e-08
6.0e-08
8.0e-08
1.0e-07
1.2e-07
1.4e-07
1.6e-07
1.8e-07
2.0e-07
2.2e-07


# __FIT SIN(F * x) * exp(decay)__

In [471]:
plt.ion()

fig = plt.figure()

def fit_func(xi, ai, bi):
    return ai*np.sin(bi * xi)
def fit_decay_rabi(xi, ai, bi, decay):
    return ai*np.sin(bi * xi)*np.exp(decay)

x = np.linspace(0, 10*np.pi, 360)

ydata = decay_data_result #5*np.sin(5*x) + np.random.normal(size=1000)    
xdata = np.linspace(0, 10*np.pi, 360)   

params, params_covariance  = scipy.optimize.curve_fit(fit_func, xdata, ydata, p0 = [5, 9])

print(params)

plt.plot(fit_func(xdata, params[0], params[1]))
plt.plot(ydata)


ValueError: operands could not be broadcast together with shapes (360,) (0,) 

In [ ]:
nwa.set_output_state("ON")
nwa.set_standard_config({"power":-50,"nop":101,"averages": 1, "cf_span":[2.5e9, 2e9], "bandwidth":10, "meas_counts":1})

ncs.enable_channel(1)
ncs.set_output_state(1,1)
ncs.set_range(1,"10mA")
ncs.set_current(1,0)
nwa.set_timeout(200000)


plt.ion()

fig = plt.figure()
z_upd = []

for i in range(10):
    ncs.set_current(1,i*1e-4)
    nwa.trig_meas()
    x_axis = nwa.get_X_axis_values()   
    data = nwa.get_data(int(nwa.get_meas_counts()))
    x_upd = x_axis#[]
    y_upd = []
    nx = 101#x_upd.shape[0]
    no_labels = 5
    step_x = int(nx/(no_labels -1))
    x_positions = np.arange(0,nx,step_x)
    x_labels = x_upd[::step_x]
    plt.yticks(x_positions, x_labels)
    
    z_upd.append(data[0][0])
    y_upd.append(i)
    plt.imshow(np.transpose(z_upd),interpolation='nearest', aspect='auto', cmap=cm.seismic, origin = 'lower')
    #nwa.set_timeout(1000)
    #plt.plot(x_axis, np.average(data[0],0))
    #line1.set_ydata(data[0][i][50])
    fig.canvas.draw()
    fig.canvas.flush_events()


In [11]:
from __future__ import division
from matplotlib.patches import Patch
from pylab import *


y = np.linspace(0,1e-3,5)
x = x_axis
X,Y = meshgrid(y, x)


Z = np.transpose(np.array(act_data))

pcolor(X, Y, Z, cmap=cm.RdBu, vmax=abs(Z).max(), vmin=-abs(Z).max())
colorbar()
#axis([-3,3,-3,3])

plt.ion()

    fig.canvas.draw()
    fig.canvas.flush_events()



In [74]:
import matplotlib.animation as animation


fig2 = plt.figure()
x = x_axis
y = [np.arange(3)]
base = np.array(act_data[0:3])
X,Y = meshgrid(x, y)
ims = []
plt.pcolormesh(X, Y, base)




In [4]:
x = np.arange(100)
y = np.arange(100)
temp = [[0]*len(x)]*len(y)
nrows, ncols = 100, 100
grid = temp #.reshape((nrows, ncols))

for i in range(len(x)):
    temp[i] = np.random.rand(100)
fig = plt.figure()
plt.imshow(grid, extent=(x.min(), x.max(), y.max(), y.min()),
           interpolation='nearest', aspect='equal', cmap=cm.jet)
plt.colorbar()

plt.ion()
x_upd = []
y_upd = np.array([])
z_upd = []

for i in range(10):    
    time.sleep(1)
    #z_upd.append([1,2,3,4])
    z_upd.append(np.random.rand(4))
    plt.imshow(np.transpose(z_upd),interpolation='nearest', aspect='auto', cmap=cm.RdBu)
    
    fig.canvas.draw()
    fig.canvas.flush_events()    



In [2]:
def serial_ports():
    """ Lists serial port names

        :raises EnvironmentError:
            On unsupported or unknown platforms
        :returns:
            A list of the serial ports available on the system
    """
    if sys.platform.startswith('win'):
        ports = ['COM%s' % (i + 1) for i in range(256)]
    elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
        # this excludes your current terminal "/dev/tty"
        ports = glob.glob('/dev/tty[A-Za-z]*')
    elif sys.platform.startswith('darwin'):
        ports = glob.glob('/dev/tty.*')
    else:
        raise EnvironmentError('Unsupported platform')

    result = []
    for port in ports:
        try:
            s = serial.Serial(port)
            s.close()
            result.append(port)
        except (OSError, serial.SerialException):
            pass
    return result


data_averaged = np.average(data,0)
peak_data = scipy.signal.find_peaks(data_averaged, height=4, distance=100)
print(peak_data[0])
print(x_axis[peak_data[0][0]])#,x_axis[peak_data[0][1]],x_axis[peak_data[0][2]], x_axis[peak_data[0][3]],x_axis[peak_data[0][4]],x_axis[peak_data[0][5]])
